In [49]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 🚀 1️⃣ Initialisation du navigateur
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://mydramalist.com/search?adv=titles&ty=68&co=2&re=2024,2024&st=3&so=relevance&page=15")
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "box")))

data = []

while True:
    # 📌 2️⃣ Récupérer la liste des dramas
    dramas = driver.find_elements(By.CLASS_NAME, "box")
    liens = [drama.find_element(By.CLASS_NAME, "block").get_attribute("href") for drama in dramas if drama.find_elements(By.CLASS_NAME, "block")]

    for lien in liens:
        driver.execute_script("window.open(arguments[0], '_blank');", lien)
        WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)  # Attendre l'ouverture d'un nouvel onglet
        time.sleep(1)  # Pause pour stabiliser l'affichage

    window_handles = driver.window_handles
    if len(window_handles) < 2:
        print("❌ Aucune nouvelle page ouverte, skipping...")
        continue

    for i in range(1, len(window_handles)):  # Ignorer l'onglet principal
        driver.switch_to.window(window_handles[i])
        time.sleep(2)

        print(f"Scraping : {driver.current_url}")  # Vérifier l'URL pour debug

        try:
            # 📌 Extraction des données avec les nouveaux XPath
            nom = driver.find_element(By.CSS_SELECTOR, "h1").text if driver.find_elements(By.CSS_SELECTOR, "h1") else "N/A"

            # 🔍 Genre
            try:
                genre_el = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "li.list-item.p-a-0.show-genres"))
                ).text
                genre=genre_el.split(": ")[1]
            except:
                genre = "N/A"

            # 🔍 Épisodes
            try:
                episodes_elements = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[2]/div/div[2]/div/div[2]/div[2]/ul/li[3]').text
                episodes=episodes_elements.split()[-1] 
            except:
                episodes = "N/A"

            # 🔍 Date de diffusion
            try:
                date_diff = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[2]/div/div[2]/div/div[2]/div[2]/ul/li[4]').text
                date_diffusion=date_diff.split(": ")[1] 
            except:
                date_diffusion = "N/A"

            # 🔍 Score
            try:
                score_el = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[2]/div/div[2]/div/div[3]/div[2]/ul/li[1]').text
                score=score_el.split(": ")[1].split(" ")[0] 
            except:
                score = "N/A"

            # 🔍 Rank
            try:
                rank_element = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[2]/div/div[2]/div/div[3]/div[2]/ul/li[2]').text
                rank = rank_element.split("#")[1]
            except:
                rank = "N/A"

            # 🔍 Téléspectateurs
            try:
                telespect = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[2]/div/div[2]/div/div[3]/div[2]/ul/li[4]').text
                telespectateurs=telespect.split(": ")[1] 
            except:
                telespectateurs = "N/A"

            # 🔍 Acteurs
            try:
                acteurs_elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div/div[2]/div/div[1]/div[4]/div[4]/ul/li/div[2]/a')
                acteurs = ", ".join([acteur.text for acteur in acteurs_elements]) if acteurs_elements else "N/A"
            except:
                acteurs = "N/A"

            # Ajouter les données à la liste
            data.append([nom, genre, episodes, date_diffusion, score, rank, telespectateurs, acteurs])
            print(f"✅ {nom} récupéré avec succès !")

        except Exception as e:
            print(f"❌ Erreur lors de l'extraction des données : {e}")

        finally:
            if len(driver.window_handles) > 1:
                driver.close()

    driver.switch_to.window(driver.window_handles[0])

    # 🔄 6️⃣ Passer à la page suivante
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.pagination-next")
        next_button.click()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "box")))
    except:
        print("🚀 Fin de la pagination.")
        break

# 📝 7️⃣ Sauvegarde en CSV
with open("dramas_page_15.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Nom", "Genre", "Épisodes", "Date de Diffusion", "Score", "Rank", "Téléspectateurs", "Acteurs"])
    writer.writerows(data)

print("\n✅ Fichier 'dramas_final.csv' créé avec succès !")

# ❌ 8️⃣ Fermer le navigateur
driver.quit()


Scraping : https://mydramalist.com/751503-just-in-time
✅ Just in Time (2024) récupéré avec succès !
Scraping : https://mydramalist.com/783450-moonlight-remains-gentle
✅ Moonlight Remains Gentle (2024) récupéré avec succès !
Scraping : https://mydramalist.com/772737-bei-xie-yi-sheng-ju-jia-hao-you-99-ci
✅ Bei Xie Yi Sheng Ju Jia Hao You 99 Ci (2024) récupéré avec succès !
Scraping : https://mydramalist.com/746813-mei-you-wo-de-cheng
✅ In the Name of the Brother (2024) récupéré avec succès !
Scraping : https://mydramalist.com/779692-prince-of-beijing-circle
✅ Prince of Beijing Circle (2024) récupéré avec succès !
Scraping : https://mydramalist.com/681845-city-of-the-city
✅ City of the City (2024) récupéré avec succès !
Scraping : https://mydramalist.com/783816-crush-the-village-husband
✅ Crush the Village Husband (2024) récupéré avec succès !
Scraping : https://mydramalist.com/36305-the-gate-of-xuan-wu
✅ The Gate of Xuan Wu (2024) récupéré avec succès !
Scraping : https://mydramalist.com